In [21]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

train_data = fetch_20newsgroups(subset='train', shuffle=True)
test_data = fetch_20newsgroups(subset='test', shuffle=True)


class_names = train_data.target_names


data = train_data.data + test_data.data
label = list(train_data.target) + list(test_data.target)

print(len(data))
print(len(label))



18846
18846


In [27]:
kf = KFold(n_splits=5)
result = []

def getDataByIndex(data, index):
    result = []
    for i in index:
        result.append(data[i])
    return result

for train_index, test_index in kf.split(data):
    train_data, train_label = getDataByIndex(data, train_index), getDataByIndex(label, train_index)
    test_data, test_label = getDataByIndex(data, test_index), getDataByIndex(label, test_index)
    assert len(train_data) == len(train_label)
    tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features=4000).fit(train_data)
    train_tfidf = tfidf.transform(train_data)
    test_tfidf = tfidf.transform(test_data)
    
    clf = LGBMClassifier()
    clf.fit(train_tfidf, train_label)
    
    pred = clf.predict(test_tfidf)
    print(pred[:10], test_label[:10])
    acc = accuracy_score(pred, test_label)
    print(acc)
    result.append(acc)
    
    

[ 7  4  4  1 14 16 13  3  2  3] [7, 4, 4, 1, 14, 16, 13, 3, 2, 4]
0.8331564986737401
[18  2  3 19 15 12 17  6  7  8] [18, 2, 3, 19, 15, 12, 17, 6, 7, 8]
0.8344388431944813
[ 0  4 17 13  3  6 14 17  6  8] [0, 4, 17, 13, 3, 6, 14, 17, 6, 8]
0.8426638365614221
[ 5 13  4  3  1  7  4 12  0 17] [5, 13, 4, 3, 1, 8, 7, 5, 0, 17]
0.8187848235606262
[ 2  8 10  0  1 15  3 10 10  9] [2, 8, 10, 0, 1, 15, 3, 10, 18, 9]
0.8201114353940037


In [29]:
avg_score = sum(result)/len(result)
print(avg_score)

0.8298310874768546
